In [51]:
from PIL import Image
import numpy as np
import math
import sys, pygame
import time
import random
PI = 3.1415926535
import os

In [52]:
class Lidar():
    def __init__(self, R, angle):
        self.R = R 
        self.sensor_data = math.inf
        self.position = [0,0]
        self.angle = angle
        
    def sense(self, drone_position, angle):
        point_to_check = []
        for i in range(1, self.R + 1):
            point = [0,0]
            check = self.change_position(i, angle)
            point[1] = check[1] + drone_position[1]
            point[0] = check[0] + drone_position[0]
            point_to_check.append(point)
        return point_to_check
    
    def change_position(self, R, angle):
        line_end_position = [0,0]
        line_end_position[1] = round(R * math.cos(angle * PI / 180))
        line_end_position[0] = round(R * math.sin(angle * PI / 180))
        return line_end_position

In [53]:
class Drone:
    def __init__(self, x, y, angle, lidars):
        self.position = [0,0]
        self.angle = angle
        self.lidars = lidars
        self.speed = 0
        self.speed_in_ms = 0
        self.score = 0
    def init_lidars(self):
        for lidar in self.lidars:
            tmp = self.change_position(lidar.R, lidar.angle)
            lidar.position[1] = self.position[1] + tmp[1]
            lidar.position[0] = self.position[0] + tmp[0]
            
    def move(self):
        self.check_angle()
        tmp = self.change_position(3, self.angle)
        self.position[1] += round(self.speed * tmp[1])
        self.position[0] += round(self.speed * tmp[0])
        for lidar in self.lidars:
            lidar.position[1] += round(self.speed * tmp[1])
            lidar.position[0] += round(self.speed * tmp[0])

        
    def change_position(self, R, angle):
        line_end_position = [0,0]
        line_end_position[1] = round((R * math.cos(angle * PI / 180)))
        line_end_position[0] = round((R * math.sin(angle * PI / 180)))
        return line_end_position
    
    def change_angle(self, direction, yaw_speed):
        self.angle += yaw_speed * direction
        self.check_angle()
        for lidar in self.lidars:   
            tmp = self.change_position(lidar.R, self.angle + lidar.angle)
            lidar.position[1] = self.position[1] + tmp[1]
            lidar.position[0] = self.position[0] + tmp[0]
        
    
    def check_angle(self):
        if self.angle >= 360:
            self.angle = 0
        if self.angle < 0:
            self.angle = 359
        
    

In [54]:
class Mods():
    def __init__(self):
        self.angle_change = 0
        self.ANGLE = 0
    def fix(self, direction):
            self.angle_change -= 1 * direction
            
    def reset(self):
        self.angle_change = self.ANGLE
        
class Red(Mods):          
    def __init__(self):
        self.front_safe_dist = 10
        self.right_safe_dist = 0 
        self.left_safe_dist = 0 
        self.acc = -1.5
        self.ANGLE = 180
        self.angle_change = self.ANGLE
        self.name = "Red"
            
class Orange(Mods):
    def __init__(self):
        self.front_safe_dist = 40 
        self.right_safe_dist = 5  
        self.left_safe_dist = 5 
        self.acc = -1
        self.ANGLE = 45
        self.angle_change = self.ANGLE 
        self.name = "Ornage"
          
class Green(Mods):    
    def __init__(self):
        self.front_safe_dist = 60
        self.right_safe_dist = 10 
        self.left_safe_dist = 10
        self.acc = 0.01
        self.ANGLE = 10
        self.angle_change = self.ANGLE
        self.name = "Green"
        
class Cruze(Mods):    
    def __init__(self):
        self.front_safe_dist = 150
        self.right_safe_dist = 0 
        self.left_safe_dist = 0
        self.acc = 0.05
        self.ANGLE = 0
        self.angle_change = self.ANGLE
        self.name = "Cruze"
        

In [63]:
class Simulator:
    def __init__(self, map_path, drone, scale):
        self.map_arr = self.png_to_arr(map_path)
        self.screen = self.init_pygame(self.map_arr.shape[0], self.map_arr.shape[1])
        self.drone = drone
        self.founed_points = []
        self.time_in_sec = 0
        self.flying = True
        self.goals_mod = False
        self.scale = scale
        self.goals = []
        
    def png_to_arr(self, map_path):
        img = Image.open(map_path)
        img=img.convert('1')
        return np.flip(np.rot90(np.asarray(img)),axis=0)
    
    def init_pygame(self, width, height):
        pygame.init()
        size = width, height
        screen = pygame.display.set_mode(size)
        return screen
    
    def render_map(self):
        for x in range(self.map_arr.shape[0]):
            for y in range(self.map_arr.shape[1]):
                if self.map_arr[x][y] == False:
                    pygame.draw.circle(self.screen,[255,0,0],[x,y],1,1)
    
    def draw_drone(self):
        self.screen.fill([255,255,255])
        pygame.draw.circle(self.screen, [255,0,0], self.drone.position, 4,4)
        for lidar in self.drone.lidars:            
            pygame.draw.line(self.screen, [0,0,0], self.drone.position , lidar.position)
            pygame.draw.circle(self.screen, [255,0,0], lidar.position, 1,1) 
            
    def find_start_pos(self):
        start_flag = False
        for y in range(self.map_arr.shape[0]):
            for x in range(self.map_arr.shape[1]):
                if self.map_arr[x,y] == True:
                    for lidar in self.drone.lidars:            
                        lidar.position = [x+5, y + 60 + 5]
                    return [x+15, y+15]
    
    def check_points(self, sensor_data, lidar):
        for point in sensor_data:      
            if self.map_arr[point[0],point[1]] == False:
                self.founed_points.append([point[0],point[1]])
                self.print_founded_points()
                lidar.sensor_data = math.hypot(point[1] - self.drone.position[1], 
                                               point[0] - self.drone.position[0])
                return
        lidar.sensor_data = math.inf
    
    def check_goals(self, sensor_data):
        for point in sensor_data:
            for goal in self.goals:
                if goal[0] == point[0] and goal[1] == point[1]:
                    self.goals.remove(goal)
                    self.drone.score += 1
                    for point in self.goals:
                        pygame.draw.circle(self.screen,[0,0,0],point,2,2)
                        pygame.display.flip() 
                    continue
                
    def print_founded_points(self):
        for edge in self.founed_points:
            pygame.draw.circle(self.screen,[255,0,0],edge,1,1)
        pygame.display.flip() 
    
    def render_map(self):
        for x in range(self.map_arr.shape[0]):
            for y in range(self.map_arr.shape[1]):
                if self.map_arr[x][y] == False:
                    pygame.draw.circle(self.screen,[255,0,0],[x,y],1,1)
    
    def crash(self):
        if not self.map_arr[self.drone.position[0]][self.drone.position[1]]:
            print("You Crashed!")
            self.flying = False
            pygame.image.save(self.screen, str(self.drone.score) + ".jpg")
        elif self.time_in_sec >= 300:
            print("Mission ended")
            self.flying = False
            pygame.image.save(self.screen, str(self.drone.score) + ".jpg")
         
    def draw_param(self):
        pygame.font.init()
        myfont = pygame.font.SysFont('Comic Sans MS', 30)
        textsurface = myfont.render(self.build_param(), False, (0, 0, 0))
        self.screen.blit(textsurface,(0,0))
        
    def build_lidars_param(self):
        i = 1
        ans = ""
        for lidar in self.drone.lidars:
            ans += "Lidar " + str(i) + ")" + \
            str("{:.1f}".format(lidar.sensor_data*self.scale)) + " "
            
            i += 1
        return ans
    
    def calc_speed_by_dis(self, position):
        return math.hypot(position[1] - self.drone.position[1],
                          position[0] - self.drone.position[0])
        
    def build_param(self):
        return "Speed:" + str("{:.2f}".format(self.drone.speed_in_ms * self.scale / 100))+'|'\
                "X:" + str(self.drone.position[0])+'|' \
                "Y:" + str(self.drone.position[1])+'|' \
                "MOD:" + self.drone.get_mod() +'|' \
                "GYRO:" + str(self.drone.angle) +'|' \
                "LIDARS:" + self.build_lidars_param() + \
                "TIME:" + str("{:.1f}".format(self.time_in_sec))+'|'\
                "SCORE:" + str(self.drone.score)
        
    def block(self):
        t_end = time.time() + 0.1
        while time.time() < t_end:
            pass     
        self.time_in_sec += 0.1
    
    def is_full_second(self, time_in_sec):
        return time_in_sec - math.floor(time_in_sec) < 0.1
    
    def process_lidars_data(self):
        for lidar in self.drone.lidars:
            point_to_check = lidar.sense(self.drone.position, lidar.angle + self.drone.angle)
            self.check_goals(point_to_check)
            self.check_points(point_to_check, lidar)
            
    def calc_speed(self, last_pos):
        if self.is_full_second(self.time_in_sec):
            self.drone.speed_in_ms = self.calc_speed_by_dis(last_pos)
            last_pos[0] = self.drone.position[0]
            last_pos[1] = self.drone.position[1] 
            
    def simulate(self):  
        self.drone.position = self.find_start_pos()
        last_pos = self.find_start_pos()
        self.drone.init_lidars()
        self.goals.append([self.drone.position[0] -1 ,self.drone.position[0] -1])
        while self.flying:
            while self.goals_mod:
                for event in pygame.event.get(): 
                    if event.type == pygame.MOUSEBUTTONDOWN:
                        if event.button == 1:  # Left mouse button.
                            if self.map_arr[event.pos[0],event.pos[1]] != False:
                                self.goals.append([event.pos[0], event.pos[1]])
                    if event.type==pygame.KEYDOWN:
                        if event.key==pygame.K_RETURN:
                            self.goals_mod = not self.goals_mod
                        if event.key==pygame.K_r:
                            t_end = time.time() + 5
                            while time.time() < t_end:
                                self.render_map()
                                pygame.display.flip()
                for point in self.goals:
                    pygame.draw.circle(self.screen,[0,0,0],point,2,2)
                pygame.display.flip() 
            for point in self.goals:
                pygame.draw.circle(self.screen,[0,0,0],point,2,2)
                pygame.display.flip() 
            #calc speed every whole second
            self.calc_speed(last_pos)
            
            #block the program for 0.1 second
            self.block()
            
            #param and drone drawing
            self.draw_drone()
            self.draw_param()
            
            #read the lidars data from the drone and save the points that discovered
            self.process_lidars_data()
            
            #print the point that discovered
            self.print_founded_points()
            
            #take an autonomous step
            self.drone.step()
            
            #end the program if the drone crash into a wall or if the mission time ends 
            #(hard coded to 5 min)
            self.crash()
            for event in pygame.event.get(): 
                #mouse handling
                if event.type == pygame.MOUSEBUTTONDOWN:
                    if event.button == 1:  # Left mouse button.
                        if self.map_arr[event.pos[0],event.pos[1]] != False:
                            self.goals.append([event.pos[0], event.pos[1]])
                            
                #keyboard handling
                if event.type==pygame.KEYDOWN:
                    #(r key)map rendering for 5 sec (block the program)
                    if event.key==pygame.K_r:
                        t_end = time.time() + 5
                        while time.time() < t_end:
                            self.render_map()
                            pygame.display.flip()
                    
                    #(esc key)quit
                    if event.key==pygame.K_ESCAPE:
                        self.flying = False
                        pygame.image.save(self.screen, str(self.drone.score) + ".jpg")
                    
                    if event.key==pygame.K_RETURN:
                        self.goals_mod = not self.goals_mod
                        
#                     if event.key==pygame.K_UP:
#                         self.drone.speed += 1 if self.drone.speed < 2.7 else 0
                        
#                     if event.key==pygame.K_DOWN:
#                         self.drone.speed -= 2
#                         if self.drone.speed <= 0:
#                             self.drone.speed = 0         

#             keys_pressed = pygame.key.get_pressed()

#             if keys_pressed[pygame.K_LEFT]:
#                 self.drone.change_angle(-1)
#                 for lidar in self.drone.lidars:
#                     point_to_check = lidar.sense(self.drone.position, lidar.angle + self.drone.angle)
#                     self.check_points(point_to_check, lidar)
#                     self.print_founded_points()
                    
#             if keys_pressed[pygame.K_RIGHT]:
#                 self.drone.change_angle(1)
#                 for lidar in self.drone.lidars:
#                     point_to_check = lidar.sense(self.drone.position, lidar.angle + self.drone.angle)
#                     self.check_points(point_to_check, lidar)
#                     self.print_founded_points()       
        pygame.quit()

In [64]:
class AutoDrone(Drone):
    
    def __init__(self, x, y, angle, lidars):
        Drone.__init__(self, x, y, angle, lidars)
        self.mods = [Red(), Orange(), Green(), Cruze()]
        self.mod = self.mods[0]
    
    def step(self):
        front = self.lidars[0].sensor_data
        right = self.lidars[1].sensor_data
        left = self.lidars[2].sensor_data
        min_dist = min((front,right,left))
        max_dist = max((front,right,left))
        self.mod = self.change_mod(self.mods, front, left, right, self.mod)
        if left > right and right < 60:
            direction = -1
        elif left < right and left < 60:
            direction = 1
        else:
            direction = random.sample((-1,1), 1)[0]
        self.speed += self.mod.acc if self.speed < 2.6 else 0
        if self.speed <= 0:
            self.speed = 0
        self.change_angle(direction,self.mod.angle_change)
        
        self.move()
        
    def get_max_dist(self, lidars):
        ans = lidars[0]
        for lidar in lidars:
            if(lidar.sensor_data>ans.sensor_data):
                ans = lidar
        return ans
    
    def get_mod(self):
        return self.mod.name
    
    def change_mod(self, mods ,front, left, right, curr_mod):
        for mod in mods:
            if front <= mod.front_safe_dist or left <= mod.left_safe_dist \
            or right <= mod.right_safe_dist:
                return mod
        return mods[len(mods) - 1]
        

In [65]:
#length in pix, angle(0=down)
lidar1 = Lidar(120, 0) 
lidar2 = Lidar(120, 45)
lidar3 = Lidar(120, 315)
lidars = [lidar1, lidar2, lidar3]

#start x, start y, start yaw, lidars list
drone = AutoDrone(0,0,0,lidars) 

# #run on single map
# #image path, drone object, scale(cm to pix)
# simulator = Simulator('image.png', drone , 2.5)

# #main loop method
# simulator.simulate() 

#run all maps in folder(folder should conatin png maps only)
directory = "folder_path"
for filename in os.listdir(directory):
    #image path, drone object, scale(cm to pix)
    simulator = Simulator(directory + '/' + filename, drone , 2.5)
    
    #main loop method
    simulator.simulate() 
    while simulator.flying:
        pass
    
        
    